In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
from tqdm import tqdm

# Configure Chrome to run in headless mode
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")  # Optional, for Windows users

In [18]:
with open("./VQ-RAD-Dataset/VQA_RAD Dataset Public.json", "r") as f:
    full_data = json.load(f)

In [19]:
filtered_data = [x for x in full_data if "not" not in x["evaluation"].lower()] 

In [20]:
len(filtered_data)

1691

In [21]:
case_urls = list(set([x["image_case_url"] for x in filtered_data]))

In [22]:
len(case_urls)

293

In [35]:
# List of URLs to scrape
# urls = [
#     "https://medpix.nlm.nih.gov/case?id=cac8c05f-3189-4dc0-8c5d-473c23fb5943"
# ]

urls = case_urls

In [36]:
results = []  # List to store dictionaries of results per URL

In [37]:
# Initialize the WebDriver with headless options
driver = webdriver.Chrome(options=chrome_options)

# Create an explicit wait object (timeout set to 10 seconds)
wait = WebDriverWait(driver, 20)

In [38]:
# Initialize the WebDriver with headless options
driver = webdriver.Chrome(options=chrome_options)

# Create an explicit wait object (timeout set to 10 seconds)
wait = WebDriverWait(driver, 20)

for url in tqdm(urls):
    driver.get(url)
    
    # Wait until the left section is present
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div#left-section")))
    except Exception as e:
        print(f"Possible issue with URL {url}: {e}")
    
    data = {}
    
    # Extract the main image URL from the left-section
    try:
        img_element = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div#left-section div.image-frame img")
        ))
        data["main_image_url"] = img_element.get_attribute("src")
    except Exception as e:
        data["main_image_url"] = None
        print(f"Error extracting main image URL from {url}: {e}")
    
    # Extract case patient details
    try:
        case_text_elem = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//div[@class='content-block image-details']")
        ))
        data["patient_details_text"] = case_text_elem.get_attribute('innerText')
        data["patient_details_html"] = case_text_elem.get_attribute('innerHTML')
    except Exception as e:
        data["patient_details_text"] = None
        data["patient_details_html"] = None
        print(f"Error extracting case details text from {url}: {e}")
    
    # Click the "Case" tab before extracting case text
    try:
        topic_tab = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[normalize-space()='Case']")))
        topic_tab.click()
        # Wait until the topic content is present
        wait.until(EC.presence_of_element_located(
            (By.XPATH, "//div[@id='image-case-tab']//div[@class='col-xs-12 col-sm-12 col-md-12 col-lg-12 content-block content-block-left']")
        ))
    except Exception as e:
        print(f"Error clicking topic tab on {url}: {e}")
    
    # Extract case text
    try:
        topic_text_elem = driver.find_element(By.XPATH, 
            "//div[@id='image-case-tab']//div[@class='col-xs-12 col-sm-12 col-md-12 col-lg-12 content-block content-block-left']")
        data["case_details_text"] = topic_text_elem.get_attribute('innerText')
        data["case_details_html"] = topic_text_elem.get_attribute('innerHTML')
    except Exception as e:
        data["case_details_text"] = None
        data["case_details_html"] = None
        print(f"Error extracting topic text from {url}: {e}")
    
    # Click the "topic" tab before extracting Topic text
    try:
        topic = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[normalize-space()='Topic']")))
        topic.click()
        # Wait until the topic content is present
        wait.until(EC.presence_of_element_located(
            (By.XPATH, "//div[@id='image-author-tab']//div[@class='col-xs-12 col-sm-12 col-md-12 col-lg-12 content-block content-block-left']")
        ))
    except Exception as e:
        print(f"Error clicking Authour tab on {url}: {e}")
    
    # Extract topic text
    try:
        topic_text_elem = driver.find_element(By.XPATH, 
            "//div[@id='image-topic-tab']//div[@class='col-xs-12 col-sm-12 col-md-12 col-lg-12 content-block content-block-left']")
        data["topic_text"] = topic_text_elem.get_attribute('innerText')
        data["topic_html"] = topic_text_elem.get_attribute('innerHTML')
    except Exception as e:
        data["topic_text"] = None
        data["topic_html"] = None
        print(f"Error extracting topic text from {url}: {e}")


    # Click the "authour" tab before extracting authour text
    try:
        authour = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[normalize-space()='Author']")))
        authour.click()
        # Wait until the topic content is present
        wait.until(EC.presence_of_element_located(
            (By.XPATH, "//div[@id='image-author-tab']//div[@class='col-xs-12 col-sm-12 col-md-12 col-lg-12 content-block content-block-left']")
        ))
    except Exception as e:
        print(f"Error clicking Authour tab on {url}: {e}")
    
    # Extract authour text
    try:
        authour_text_elem = driver.find_element(By.XPATH, 
            "//div[@id='image-author-tab']//div[@class='col-xs-12 col-sm-12 col-md-12 col-lg-12 content-block content-block-left']")
        data["authour_text"] = authour_text_elem.get_attribute('innerText')
        data["authour_html"] = authour_text_elem.get_attribute('innerHTML')
    except Exception as e:
        data["authour_text"] = None
        data["authour_html"] = None
        print(f"Error extracting topic text from {url}: {e}")
    
    results.append(data)

    with open("./VQ-RAD-Case-Base/results.json", "w") as f:
        json.dump(results, f)

driver.quit()

  1%|▊                                                                                 | 3/293 [00:05<08:09,  1.69s/it]

Error extracting topic text from https://medpix.nlm.nih.gov/case?id=116aa908-f0a4-47e6-bbaf-8269a1634111: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='image-topic-tab']//div[@class='col-xs-12 col-sm-12 col-md-12 col-lg-12 content-block content-block-left']"}
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7673AC6A5+28789]
	(No symbol) [0x00007FF767315B20]
	(No symbol) [0x00007FF7671A8F9A]
	(No symbol) [0x00007FF7671FF346]
	(No symbol) [0x00007FF7671FF57C]
	(No symbol) [0x00007FF767252B17]
	(No symbol) [0x00007FF76722736F]
	(No symbol) [0x00007FF76724F7E3]
	(No symbol) [0x00007FF767227103]
	(No symbol) [0x00007FF7671EFFC0]
	(No symbol) [0x00007FF7671F1273]
	GetHandleVerifier [0x00007FF7676F1AED+3458237]
	GetHandleVerifier [0x00007FF76770829C+3550316]
	GetHandleVer

  2%|█▉                                                                                | 7/293 [00:11<07:26,  1.56s/it]

Error extracting topic text from https://medpix.nlm.nih.gov/case?id=607675c1-11c8-46f1-8389-7ff55513b0ae: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='image-topic-tab']//div[@class='col-xs-12 col-sm-12 col-md-12 col-lg-12 content-block content-block-left']"}
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7673AC6A5+28789]
	(No symbol) [0x00007FF767315B20]
	(No symbol) [0x00007FF7671A8F9A]
	(No symbol) [0x00007FF7671FF346]
	(No symbol) [0x00007FF7671FF57C]
	(No symbol) [0x00007FF767252B17]
	(No symbol) [0x00007FF76722736F]
	(No symbol) [0x00007FF76724F7E3]
	(No symbol) [0x00007FF767227103]
	(No symbol) [0x00007FF7671EFFC0]
	(No symbol) [0x00007FF7671F1273]
	GetHandleVerifier [0x00007FF7676F1AED+3458237]
	GetHandleVerifier [0x00007FF76770829C+3550316]
	GetHandleVer

 14%|███████████                                                                      | 40/293 [01:05<06:55,  1.64s/it]

...

 86%|█████████████████████████████████████████████████████████████████████           | 253/293 [21:24<01:11,  1.78s/it]

Error extracting topic text from https://medpix.nlm.nih.gov/case?id=4726c455-b566-4931-aecb-41ed165e1bfa: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='image-topic-tab']//div[@class='col-xs-12 col-sm-12 col-md-12 col-lg-12 content-block content-block-left']"}
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7673AC6A5+28789]
	(No symbol) [0x00007FF767315B20]
	(No symbol) [0x00007FF7671A8F9A]
	(No symbol) [0x00007FF7671FF346]
	(No symbol) [0x00007FF7671FF57C]
	(No symbol) [0x00007FF767252B17]
	(No symbol) [0x00007FF76722736F]
	(No symbol) [0x00007FF76724F7E3]
	(No symbol) [0x00007FF767227103]
	(No symbol) [0x00007FF7671EFFC0]
	(No symbol) [0x00007FF7671F1273]
	GetHandleVerifier [0x00007FF7676F1AED+3458237]
	GetHandleVerifier [0x00007FF76770829C+3550316]
	GetHandleVer

 92%|█████████████████████████████████████████████████████████████████████████▍      | 269/293 [21:54<00:45,  1.91s/it]

Error extracting topic text from https://medpix.nlm.nih.gov/case?id=55077fdb-219e-427d-83c9-af608877f918: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='image-topic-tab']//div[@class='col-xs-12 col-sm-12 col-md-12 col-lg-12 content-block content-block-left']"}
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7673AC6A5+28789]
	(No symbol) [0x00007FF767315B20]
	(No symbol) [0x00007FF7671A8F9A]
	(No symbol) [0x00007FF7671FF346]
	(No symbol) [0x00007FF7671FF57C]
	(No symbol) [0x00007FF767252B17]
	(No symbol) [0x00007FF76722736F]
	(No symbol) [0x00007FF76724F7E3]
	(No symbol) [0x00007FF767227103]
	(No symbol) [0x00007FF7671EFFC0]
	(No symbol) [0x00007FF7671F1273]
	GetHandleVerifier [0x00007FF7676F1AED+3458237]
	GetHandleVerifier [0x00007FF76770829C+3550316]
	GetHandleVer

100%|████████████████████████████████████████████████████████████████████████████████| 293/293 [22:35<00:00,  4.63s/it]


In [39]:
# Print all results
# for result in results:
#     print(result)

In [55]:
updated_result = {}

In [56]:
for url, result in zip(urls, results):
    result={"case_url":url, **result}
    updated_result[url] = result

In [57]:
with open("./VQ-RAD-Case-Base/results.json", "w") as f:
        json.dump(updated_result, f)

In [50]:
results[1].keys()

dict_keys(['main_image_url', 'patient_details_text', 'patient_details_html', 'case_details_text', 'case_details_html', 'topic_text', 'topic_html', 'authour_text', 'authour_html'])

In [ ]:
# https://medpix.nlm.nih.gov/rest/image.json?imageID=28c37d09-9635-427e-98b9-385b1370be1d&withencounter=true&withtopic=true&withcme=true
# ?image_id rest api info to collect info

In [54]:
[x["topic_text"][:10] if x["topic_text"] != None else None for x in results ]

['VIEW TOPIC',
 'VIEW TOPIC',
 None,
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 None,
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 None,
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 None,
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC',
 'VIEW TOPIC

In [42]:
results[-1].keys()

dict_keys(['main_image_url', 'patient_details_text', 'patient_details_html', 'case_details_text', 'case_details_html', 'topic_text', 'topic_html', 'authour_text', 'authour_html'])

In [ ]:
# result[list(result.keys())[8]]

In [58]:
len(urls)

293

In [70]:
query_ids = [x.split("case?id=")[-1] for x in urls]

In [83]:
query_dict = {}

In [84]:
import requests

def fetch_json_data(url):
    """
    Fetch JSON data from the given URL and return it as a Python dictionary.
    
    Parameters:
        url (str): The URL to fetch JSON data from.
    
    Returns:
        dict: The JSON data as a dictionary if successful, otherwise None.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return None

# Example usage:
# url = "https://medpix.nlm.nih.gov/rest/image.json?imageID=5d6f9f35-a9db-411b-bdf0-23530197db74&withencounter=true&withtopic=true&withcme=true"
# data = fetch_json_data(url)
# print(data)


In [85]:
# query_ids

In [ ]:
for query in tqdm(query_ids):
    query_url = f"https://medpix.nlm.nih.gov/rest/image.json?imageID={query}&withencounter=true&withtopic=true&withcme=true"
    json_data = fetch_json_data(query_url)
    query_dict[f"https://medpix.nlm.nih.gov/case?id={query}"] = json_data

 19%|███████████████▊                                                                 | 57/293 [00:30<02:02,  1.93it/s]

Error fetching data from https://medpix.nlm.nih.gov/rest/image.json?imageID=cb2a29bc-5c4f-4a56-991d-1a309ec09507&withencounter=true&withtopic=true&withcme=true: Expecting value: line 1 column 1 (char 0)


 37%|█████████████████████████████▏                                                  | 107/293 [00:56<01:36,  1.93it/s]

Error fetching data from https://medpix.nlm.nih.gov/rest/image.json?imageID=cb2a29bc-5c4f-4a56-991d-1a309ec09505&withencounter=true&withtopic=true&withcme=true: Expecting value: line 1 column 1 (char 0)


 44%|██████████████████████████████████▉                                             | 128/293 [01:07<01:27,  1.90it/s]

...

 58%|██████████████████████████████████████████████▏                                 | 169/293 [01:29<00:58,  2.11it/s]

Error fetching data from https://medpix.nlm.nih.gov/rest/image.json?imageID=cb2a29bc-5c4f-4a56-991d-1a309ec09508&withencounter=true&withtopic=true&withcme=true: Expecting value: line 1 column 1 (char 0)


 63%|██████████████████████████████████████████████████▏                             | 184/293 [01:37<00:53,  2.05it/s]

Error fetching data from https://medpix.nlm.nih.gov/rest/image.json?imageID=f080986d-af2a-477f-988a-adb753d30832&withencounter=true&withtopic=true&withcme=true: Expecting value: line 1 column 1 (char 0)


 63%|██████████████████████████████████████████████████▊                             | 186/293 [01:38<00:56,  1.89it/s]

In [ ]:
with open("./VQ-RAD-Case-Base/structured_results.json", "w") as f:
        json.dump(query_dict, f)

In [ ]:
query_dict

In [ ]:
https://medpix.nlm.nih.gov/rest/image.json?imageID=5d6f9f35-a9db-411b-bdf0-23530197db74&withencounter=true&withtopic=true&withcme=true

In [1]:
with open("./VQ-RAD-Case-Base/structured_results.json", "r") as f:
        query_dict = json.load(f)

In [2]:
len(query_dict)

293